## CS143 Final Project Code Appendix
**Fall 2020, 
Written by Daniel Cox

This notebook contains python code to instatiate and demonstrate a prototype    
blockchain to be used with our "Votechain" voting system.  


It includes the following:


**Classes**
* Block
* Blockchain

**Functions**
* create_key_pair
* generate_signature
* verify_signature
* create_voter_key_dictionary

**Demonstrations**
* Creating fake voters, assigning them public and private keys and creating an empty Votechain
* Adding vote blocks to the votechain, one for each voter
* Printing out each voter's password and vote
* Verifying the signatures on all blocks with the public key of each block.
* Verifying the hashes on all blocks
* Verifying that the timestamps on all blocks are before the end of the election period
* Tallying Votes
* Trying to illegally change the vote on a block 
* Trying to illegally change the signature on a block
* Trying to illegally change the password on a block
* Trying to illegally change the timestamp on a block
* Finding a block by its public key
* Viewing a particular block
* Validating a particular block
* Stripping all public keys from the votechain

### Imported python packages

In [ ]:
! pip install pycrypto

In [ ]:
import numpy as np
import hashlib
import json
import time
from datetime import datetime
from Crypto.PublicKey import RSA
from hashlib import sha512
import pandas as pd

### Classes

In [ ]:

# Class to represent a voting transaction as a block
class Block():
    def __init__(self, index, password, signature, public_key, vote):
        self.index = index
        self.password = password
        self.timestamp = time.time()
        self.vote = vote
        self.signature = signature
        self.public_key = public_key
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(
            str(self.index).encode('utf-8') + str(self.timestamp).encode('utf-8') + self.password.encode('utf-8') \
            + str(self.signature).encode('utf-8') + str(self.vote).encode('utf-8'))

    def check_signature(self, public_key=None):
        if public_key == None:
            public_key = self.public_key
        hash = int.from_bytes(sha512(self.password.encode('utf-8')).digest(), byteorder='big')
        hashFromSignature = pow(self.signature, public_key[1], public_key[0])
        if hash == hashFromSignature:
            return True
        else:
            return False

    def check_hash(self, hash=None):
        if hash == None:
            hash = self.hash
        if hash.hexdigest() == self.calculate_hash().hexdigest():
            return True
        else:
            return False
        
    def check_timestamp(self, end_time, mask='%m/%d/%Y %I:%M%p'):
        end = datetime.strptime(end_time, mask)
        vote = datetime.fromtimestamp(self.timestamp)
        timediff = end - vote
        if timediff.total_seconds() >= 0:
            return True
        else:
            return False

# Class to represent a voting Blockchain
class Blockchain():
    def __init__(self):
        self.chain_index = 0
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, 'none', 'none', 'none', 'none')

    def get_last_block(self):
        return self.chain[-1]

    def add_block(self, password, signature, public_key, vote):
        index = self.chain_index + 1
        new_block = Block(index, password, signature, public_key, vote)
        self.chain.append(new_block)
        self.chain_index += 1

    def check_hashes(self):
        bad_hashes = []
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            if current_block.check_hash() == False:
                bad_hashes.append(i)
        if bad_hashes:
            print(f'The following block\'s hashes are invalid {bad_hashes}')
        else:
            print('All hashes valid')

    def check_signatures(self):
        bad_signatures = []
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            if current_block.check_signature() == False:
                bad_signatures.append(i)
        if bad_signatures:
            print(f'The following block\'s signatures are invalid {bad_signatures}')
        else:
            print('All signatures valid')
            
    def check_timestamps(self, end_time):
        bad_timestamps = []
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            if current_block.check_timestamp(end_time) == False:
                bad_timestamps.append(i)
        if bad_timestamps:
            print(f'The following block\'s timestamps are invalid {bad_signatures}')
        else:
            print('All timestamps valid')

    def list_votes(self): 
        votes = {}
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            current_key = current_block.signature
            votes[current_key] = current_block.vote
        return list(votes.values())

    def count_votes(self): # if multiple votes from the same voter, counts only last
        votes = pd.Series(self.list_votes()).value_counts()
        for idx in votes.index:
            print(idx, votes[idx])

    def find_block(self, public_key):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            if public_key == current_block.public_key:
                print(f'The desired block is block {current_block.index}')
                return
        print(f'public key {public_key} not found')

    def view_block(self, index_number):
        block = self.chain[index_number]
        print(f'Index number: {block.index}')
        print(f'Password: {block.password}')
        print(f'Public_key: {block.public_key}')
        print(f'Time_stamp: {block.timestamp}')
        print(f'Vote: {block.vote}')

    def validate_block(self, index_number, end_time):
        block = self.chain[index_number]
        if block.check_hash() == False:
            print('Invalid Hash')
        else:
            print('Valid Hash')
        if block.check_signature() == False:
            print('Invalid Signature')
        else:
            print('Valid Signature')
        if block.check_timestamp(end_time) == False:
            print('Invalid timestamp')
        else:
            print('Valid timestamp')
            
    def strip_public_keys(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            current_block.public_key = 0
        print(f'All public keys set to 0')
        



### Functions

In [ ]:
# Generate a public/private key pair
def create_key_pair():
    keyPair = (RSA.generate(bits=1024))
    private_key = (keyPair.n , keyPair.d)
    public_key = (keyPair.n , keyPair.e)
    return (private_key , public_key)

# Convert a name to a signature using a private key
def generate_signature(name, private_key):
    hash = int.from_bytes(sha512(name.encode('utf-8')).digest(), byteorder='big')
    signature = pow(hash, private_key[1], private_key[0])
    return signature


# Check if a signature matches a name using a public key
def verify_signature(name, signature, public_key):
    hash = int.from_bytes(sha512(name.encode('utf-8')).digest(), byteorder='big')
    hashFromSignature = pow(signature, public_key[1], public_key[0])
    if hash == hashFromSignature:
        return True
    else:
        return False

# Given a list of voters names, create a voter dictionary with keys equal to the voter's name
# and within each dictionary another dictionary with keys = 'private_key' and 'public_key'
def create_voter_key_dictionary(voters):
    voters_dictionary = {}
    for i in range(len(voters)):
        voter_keys = {}
        key_pair = create_key_pair()
        voter_keys['private_key'] = key_pair[0]
        voter_keys['public_key'] = key_pair[1]
        voters_dictionary[voters[i]] = voter_keys
    return voters_dictionary


### Demonstrations

*Creating fake voters assigning them public and private keys and creating an empty Votechain*

In [ ]:
# Creating a list of voter passwords
voters = ['XXC54372', 'YYYF651_22', 'A4326dEFt7', 'BBC&&Djk2', '111_01-3AB', 'NNNICE_043', 'Howiz76234',
         'LB_454545', 'Sam_Great5', 'Holy_6632z']

possible_votes = ['Biden','My Dog'] # — will randomly assign votes

# Giving each voter a public/private key pair and storing them in a dictionary along with the voter's name
# This would be done when a voter registers to vote
voter_dict = create_voter_key_dictionary(voters)

# Creating a Blockchain instance
voter_blockchain = Blockchain()
print('Voter passwords')
for p in voters:
    print(p)

Voter passwords
XXC54372
YYYF651_22
A4326dEFt7
BBC&&Djk2
111_01-3AB
NNNICE_043
Howiz76234
LB_454545
Sam_Great5
Holy_6632z


*Adding vote blocks to the votechain, one for each voter*

In [ ]:
for password in voter_dict.keys():
    signature  = generate_signature(password,  voter_dict[password]['private_key'])
    voter_blockchain.add_block(password, signature, voter_dict[password]['public_key'], np.random.choice(possible_votes))
print(f'The votechain now has {len(voter_blockchain.chain)-1} vote blocks')

The votechain now has 10 vote blocks


*Printing out each voter's password and vote*

In [ ]:
for block in voter_blockchain.chain[1:]:
    print(block.password, block.vote)

XXC54372 Biden
YYYF651_22 My Dog
A4326dEFt7 Biden
BBC&&Djk2 Biden
111_01-3AB Biden
NNNICE_043 Biden
Howiz76234 Biden
LB_454545 My Dog
Sam_Great5 My Dog
Holy_6632z My Dog


*Verifying the signatures on all blocks with the public key of each block*

In [ ]:
voter_blockchain.check_signatures()

All signatures valid


*Verifying that the data on all blocks has not been changed since the original vote block was created*

In [ ]:
voter_blockchain.check_hashes()

All hashes valid


*Verifying that the timestamps on all blocks are before the end of the election period*

In [ ]:
end_time = '12/17/2020 11:59PM'
voter_blockchain.check_timestamps(end_time)

All timestamps valid


*Counting Votes*

In [ ]:
voter_blockchain.count_votes()

Biden 6
My Dog 4


*Trying to illegally change the vote on block 1*

In [ ]:
voter_blockchain.chain[1].vote = 'Dan Cox'
voter_blockchain.check_hashes()

The following block's hashes are invalid [1]


*Trying to illegally change the signature on block 3*

In [ ]:
voter_blockchain.chain[3].signature= voter_blockchain.chain[2].signature
voter_blockchain.check_signatures()

The following block's signatures are invalid [3]


*Trying to illegally change the password on block 3*

In [ ]:
voter_blockchain.chain[3].name = 'XX437p5'
voter_blockchain.check_hashes()

The following block's hashes are invalid [1, 3]


Trying to illegally change the timestamp on block 5

In [ ]:
voter_blockchain.chain[5].timestamp= time.time()
voter_blockchain.check_hashes()

The following block's hashes are invalid [1, 3, 5]


*Finding block 5 by its public key*

In [ ]:
voter_blockchain.find_block(voter_blockchain.chain[5].public_key)

The desired block is block 5


*Viewing block 6*

In [ ]:
voter_blockchain.view_block(6)

Index number: 6
Password: NNNICE_043
Public_key: (145320634893847971220258739551424099461769113186518571211807453831773133407457358464550145670378923867962831782425314118053848990529231333348444095701345822122049224733643794960028922963308682150973103373559412516829382769807101454258535972823484169284022555573902889860946777953281253949076227300589634748571, 65537)
Time_stamp: 1608240639.0666656
Vote: Biden


*Validating block 6*

In [ ]:
voter_blockchain.validate_block(6, '07/11/2021 02:45PM')

Valid Hash
Valid Signature
Valid timestamp


*Stripping all public keys from the votechain*

In [ ]:
voter_blockchain.strip_public_keys()
print(' ')
print('Showing block 1:\n')
voter_blockchain.view_block(1)

All public keys set to 0
 
Showing block 1:

Index number: 1
Password: XXC54372
Public_key: 0
Time_stamp: 1608240639.041016
Vote: Dan Cox
